<h3>En estas dos celdas se genera el método para recolectar los tuits y almacenarlos en un .csv</h3>

In [1]:
import glob
import itertools
import pandas as pd
import snscrape.modules.twitter as sntwitter

column_names = ['user', 'date', 'content', 'user_location']
dftemp = pd.DataFrame(columns = column_names)
loc = '19.3206553, -99.1526775, 40km'

terms = ["COVID", "COVID19", "COVID-19", "Coronavirus", "Pandemia", "Cubrebocas", "Sana Distancia","#COVID", "#COVID19",
         "#COVID-19", "#Coronavirus", "#Pandemia", "#Cubrebocas", "#SanaDistancia", "#Delta", "AMLO", "#AstraZeneca", 
        "#Vacuna", "#Vacunación", "#Pfizer", "#Biontech", "Vacuna", "Vacunación", "Pfizer", "vacuna covid", "#vacunacovid",
        "Astra Zeneca", "#Moderna", "#Vacunate", "#LavadoDeManos", "#UsaCubrebocas", "Janssen", "#Janssen", "#johnsonandjohnson",
        "Sputnik V", "#SputnikV", "Cansino", "#Cansino", "Sinovac", "#Sinovac", "Omicron", "#Omicron",
        "test", "tests", "cuarentena", "quinta ola", "#quintaola"]

In [ ]:
desde = '2022-03-08'
hasta = '2022-03-15'
for item in terms:
    try:
        df = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(item + ' lang:es since:' +  desde + ' until:' + hasta + ' geocode:"{}"'.format(loc)).get_items(), 500))[['user', 'date', 'content']]
        df['user_location'] =  df['user'].apply(lambda x: x['location'])
    except KeyError:
        df = pd.DataFrame(columns = column_names)

    dftemp = pd.concat([dftemp, df], axis=0)
    print(item)

df2 = dftemp[['date', 'content', 'user_location']]
df2.to_csv("C://Tweets_2022.csv", sep=',', index=False)

<h3>En esta celda se realiza el proceso de leer los todos los archivos con tuits que se descargaron, se concentran en un mismo dataframe, y se eliminan todos aquellos que son repetidos, preservando el primero. esto genera un archivo maestro con todos los tuits recolectados</h3>

In [ ]:
ruta = "C://NewFiles/*.csv"
df = pd.concat(map(pd.read_csv, glob.glob(ruta)))

dfsorted = df.sort_values(by=['content'])
dfdup = dfsorted[dfsorted.duplicated('content')]

dfdup.drop_duplicates(subset ="content", keep = 'first', inplace = True)

dfdup['fecha'] = pd.to_datetime(dfdup['date'])

dfinal = dfdup.iloc[: , 1:]

dfinal = dfinal.sort_values(by="fecha")

dfinal.reindex(columns=["fecha", "content", "user_location"]).to_csv('C://Concentrado2.csv', index=False,header=False)

<h3>A continuación, se toman todos los tuits y se construye la serie de tiempo del indicador diario del sentimiento agrupado</h3>

In [ ]:
import pandas as pd
from textblob import TextBlob
from pysentimiento import create_analyzer

ruta = 'C://Concentrado2.csv'

df = pd.read_csv(ruta, header=None)
df.columns = ["Fecha", "Tweet","User_location"]
df['Tweet'] = df['Tweet'].str.translate({ord(c): " " for c in "!@#$%^&*,.()""[]{}';:/<>?\|`’~-=_+"})
df['Tweet'] = df['Tweet'].str.replace('\n',' ')

analyzer = create_analyzer(task="sentiment", lang="es")

def getsent(sentscore):
    valor = sentscore.output
    #print(valor)
    if valor == 'POS':
        return sentscore.probas[valor]
    elif valor == 'NEU':
        return sentscore.probas[valor] * .5
    else:
        return sentscore.probas[valor] * -1

def sentpol(texto):
    salida = analyzer.predict(texto)
    return getsent(salida)

df['Polarity'] = df.apply(lambda row: sentpol(row['Tweet']), axis=1)

df1 = df[['Fecha','User_location','Polarity']]

df1 = df1.set_index("Fecha")

df1 = df1.reset_index(level=0)

df2 = df1[['Fecha', 'Polarity']]
df3 = df2.groupby(['Fecha']).mean()

df3= df3.reset_index(level=0)

df3.to_csv("C://SentpolAgrupado2.csv", index=False)